## Simulation of a M/M/1 queue using processes

In [1]:
using SimJulia
using Distributions
using RandomStreams
using Distributions

const SEED = 12345

rand_dist(rng::MRG32k3a, Dist::Distribution) = quantile(Dist, rand(rng))

seeds = [SEED, SEED, SEED, SEED, SEED, SEED]
gen = MRG32k3aGen(seeds)

Seed for next MRG32k3a generator:
[12345,12345,12345,12345,12345,12345]

In [2]:
include("tally.jl")

stdev (generic function with 1 method)

In [3]:
include("tallystore.jl")

add (generic function with 2 methods)

In [4]:
println("M/M/1 with processes")

#rates
λ = 1.9
μ = 2.0
ρ = λ/μ

type System
    W
    arrival
    service
    counter
    arrgen
    servgen
    
    function System(text::String)
        s = new()
        s.W = Tally(text)
        s.arrival = Exponential(1.0/λ)
        s.service = Exponential(1.0/μ)
        s.arrgen = next_stream(gen)
        s.servgen = next_stream(gen)
        return s;
    end
end

function restart(s::System)
    init(s.W)
    next_substream!(s.arrgen)
    next_substream!(s.servgen)
end

M/M/1 with processes


restart (generic function with 1 method)

In [5]:
s = System("Waiting Times")

System(Tally("Waiting Times",0,0.0,Inf,-Inf,0.0,0.0,0.0),Distributions.Exponential{Float64}(θ=0.5263157894736842),Distributions.Exponential{Float64}(θ=0.5),#undef,Full state of MRG32k3a generator:
Cg = [12345,12345,12345,12345,12345,12345]
Bg = [12345,12345,12345,12345,12345,12345]
Ig = [12345,12345,12345,12345,12345,12345],Full state of MRG32k3a generator:
Cg = [3692455944,1366884236,2968912127,335948734,4161675175,475798818]
Bg = [3692455944,1366884236,2968912127,335948734,4161675175,475798818]
Ig = [3692455944,1366884236,2968912127,335948734,4161675175,475798818])

In [6]:
# Allow a simulation with a fixed number of clients
function source(env::Environment, s::System, counter::Resource, limit::Float64)
    i = 0
    while (true)
        yield(Timeout(env, rand_dist(s.arrgen, s.arrival)))
        if (now(env) > limit) break end
        i += 1
        Process(env, customer, s, i, counter)
    end
end

function source_fixed(env::Environment, s::System, counter::Resource, nCusts::Int64)
    for i = 1:nCusts
        yield(Timeout(env, rand_dist(s.arrgen, s.arrival)))
        Process(env, customer, s, i, counter)
    end
end

function customer(env::Environment, s::System, idx::Int, counter::Resource)
    # Record the arrival time in the system
    arrive = now(env)
    yield(Request(counter))
    # The simulation clock now contains the time when the client goes to the server.
    wait = now(env) - arrive
    # Record the waiting time
    add(s.W, wait)
    yield(Timeout(env, rand_dist(s.servgen, s.service)))
    yield(Release(counter))
end

customer (generic function with 1 method)

In [7]:
function onesim(s::System)    
    env = Environment()
    s.counter = Resource(env, 1)
    
    Process(env, source_fixed, s, s.counter, 10000)
    run(env)
end

onesim (generic function with 1 method)

In [8]:
onesim(s)

In [9]:
average(s.W)

8.630432848122487

In [10]:
variance(s.W)

58.047664581853155

In [11]:
meanWaits = TallyStore("Temps d'attente moyen")

TallyStore(Float64[],Tally("Temps d'attente moyen",0,0.0,Inf,-Inf,0.0,0.0,0.0))

In [12]:
Nobs = 100

100

In [13]:
for n=1:Nobs
    restart(s)
    onesim(s)
    w = average(s.W)
    N = nobs(s.W)
    add(meanWaits, w)
    println("Sim $n. Average waiting time: $w. Number of observations: $N")
end

Sim 1. Average waiting time: 7.550875546475624. Number of observations: 10000
Sim 2. Average waiting time: 6.135640065836502. Number of observations: 10000
Sim 3. Average waiting time: 18.479700395704146. Number of observations: 10000
Sim 4. Average waiting time: 7.1030035604970845. Number of observations: 10000
Sim 5. Average waiting time: 8.403267787576848. Number of observations: 10000
Sim 6. Average waiting time: 6.415659416467969. Number of observations: 10000
Sim 7. Average waiting time: 8.73956120790153. Number of observations: 10000
Sim 8. Average waiting time: 13.827590833432248. Number of observations: 10000
Sim 9. Average waiting time: 7.2529369795613725. Number of observations: 10000
Sim 10. Average waiting time: 6.972471705468727. Number of observations: 10000
Sim 11. Average waiting time: 25.000757727815806. Number of observations: 10000
Sim 12. Average waiting time: 10.403505705804962. Number of observations: 10000
Sim 13. Average waiting time: 6.573875031066891. Number 

In [14]:
average(meanWaits.t)

9.485065765462775

In [15]:
variance(meanWaits.t)

14.333654709875363

In [23]:
stdev(meanWaits.t)

3.7859813404024276

In [16]:
var(meanWaits.obs)

14.333654709875377

In [17]:
function ci_normal(n::Int64, mean::Float64, stdev::Float64, α::Float64)
    z = quantile(Normal(), 1-α/2)
    w = z*stdev/sqrt(n)
    # Lower bound
    l = mean - w
    # Upper bound
    u = mean + w
    
    return l, u
end

ci_normal (generic function with 1 method)

In [18]:
ci_normal(nobs(meanWaits.t), average(meanWaits.t), stdev(meanWaits.t), 0.05)

(8.74302705812983,10.22710447279572)

In [19]:
function ci_tdist(n::Int64, mean::Float64, stdev::Float64, α::Float64)
    z = quantile(TDist(n-1), 1-α/2)
    w = z*stdev/sqrt(n)
    # Lower bound
    l = mean - w
    # Upper bound
    u = mean + w
    
    return l, u
end

ci_tdist (generic function with 1 method)

In [20]:
ci_tdist(nobs(meanWaits.t)-1, average(meanWaits.t), stdev(meanWaits.t), 0.05)

(8.729965100548558,10.240166430376991)

Average waiting time according to M/M/1 formulas. In stationnary regime, the mean waiting time is
$$ \overline{W} = \frac{\frac{\rho}{\mu}}{1-\rho} = \frac{\rho}{\mu-\lambda}. $$

In [21]:
w = (ρ/μ)/(1-ρ)

9.499999999999991

In [22]:
sortedObs = sort(meanWaits.obs)

100-element Array{Float64,1}:
  4.30956
  4.33347
  5.25629
  5.28772
  5.61061
  5.79686
  5.85531
  5.93481
  5.94605
  6.01314
  6.09921
  6.13564
  6.2245 
  ⋮      
 12.988  
 13.8276 
 14.236  
 15.5214 
 15.6721 
 17.4408 
 18.4724 
 18.4797 
 19.2527 
 19.7818 
 23.3987 
 25.0008 

Bootstrap?

In [ ]:
# Construct a bootstrap sample
unif=next_stream(gen)

In [ ]:
m = 200

wboostrap = TallyStore("Boostrap estimator")

In [ ]:
for j = 1:m
    xb = Array(Float64,Nobs)
    for i=1:Nobs
        k = Int64(ceil(rand(unif)*Nobs))
        xb[i] = meanWaits.obs[k]
    end
    add(wboostrap, mean(xb))
end

In [ ]:
average(wboostrap.t)

In [ ]:
sortwb = sort(wboostrap.obs)

In [ ]:
stdev(meanWaits.t)

In [ ]:
α = 0.05
ceil(m*(1-α/2))

In [ ]:
ceil(m*α/2)

In [ ]:
y = average(meanWaits.t)

In [ ]:
2*y-sortwb[195]

In [ ]:
2*y-sortwb[5]